In [338]:
# MIT License
# 
# Copyright (c) 2024 [저작권자 이름]
# 
# This software is released under the MIT License.
# https://opensource.org/licenses/MIT


In [337]:
from pyhwpx import Hwp
import win32com.client as win32
import pandas as pd
hwp = Hwp()
hwp.Open(r"./template2.hwp") # 주간행사계획 한글파일 상대경로

True

In [309]:
df = pd.read_excel("eventdummy.xlsx")
#엑셀에서 더미데이터 read
df = df.replace(r'\n', '', regex=True)
# 데이터안에 \n 다지우기

df = df.fillna(" ")  # NaN 값을 으로 채우기
df.tail()

,date,time,eventname,place,personnel,department,note
14,4. 13.(토),10:00:00,·2024년 특성화 사업 제1회차‘인문학 콘서트’,유성문화원 1층전시실,30명,문화관광과,
15,4. 13.(토),10:00:00,·행복한 문화학교 「햄스터 로봇 코딩」,유성도서관,14명,도서관운영과,
16,4. 13.(토),15:00:00,·노은도서관 동화 들려주기(Story Time),노은도서관,40명,도서관운영과,
17,4. 13.(토),15:00:00,·독서프로그램 「귀 기울여 영어동화」 영어원서 읽기,어린이영어마을도서관,10명,도서관운영과,
18,4. 14.(일),,,,,,


In [310]:
date_counts = df['date'].value_counts()
date_counts

date
4. 13.(토)     6
4. 11.(목)     4
4. 8.(월)      3
4. 12. (금)    2
4. 9.(화)      2
4. 10.(수)     1
4. 14.(일)     1
Name: count, dtype: int64

In [320]:
import re

date_counts = df['date'].value_counts().reset_index()
date_counts.columns = ['date', 'count']  # 열 이름 지정

# 월과 일만 추출하는 함수 정의
def extract_month_day(date_str):
    match = re.search(r'(\d+)\.\s*(\d+)', date_str)
    if match:
        return int(match.group(1)), int(match.group(2))  # 월, 일 반환
    return None

# 'month', 'day' 열 추가
date_counts[['month', 'day']] = date_counts['date'].apply(lambda x: pd.Series(extract_month_day(x)))

# 월과 일 기준으로 정렬
date_counts = date_counts.sort_values(by=['month', 'day']).drop(columns=['month', 'day']).reset_index(drop=True)

#date 컬럼만
date_column_df = date_counts[['date']]

# 정렬된 결과 출력
print(date_counts)

         date  count
0    4. 8.(월)      3
1    4. 9.(화)      2
2   4. 10.(수)      1
3   4. 11.(목)      4
4  4. 12. (금)      2
5   4. 13.(토)      6
6   4. 14.(일)      1


In [312]:
# print(date_counts.iloc[0])
# count_value = date_counts.iloc[0]['count']
# print(count_value)
#데이터 갯수만큼 열 만들기

for index, row in date_counts.iterrows():
    pset = hwp.HParameterSet.HTableDeleteLine
    hwp.move_to_field(f'time{{{{{index}}}}}')
    print(index)
    count = row['count']
    
    if count>2 :
        for i in range(count - 2):
            hwp.TableAppendRow()
            print(f"{index}" + ": append" )
    elif (count == 2):
        continue
    else :
        hwp.TableLowerCell()
        hwp.HAction.GetDefault("TableDeleteRow", pset.HSet)
        #TLqkf
        hwp.HAction.Execute("TableDeleteRow", pset.HSet)
        print(f"{index}"+": delete" )
        
        
    

0
0: append
1
2
2: delete
3
3: append
3: append
4
5
5: append
5: append
5: append
5: append
6
6: delete


date컬럼에 date값들 집어넣는 코드    (주의사항 : 한글의 월일 column에 셀 필드값이 date로 되어야함)

In [313]:
times = (df['time'])
for index,time in enumerate(times):
    print(index , time)
    hwp.put_field_text("time",f"{time}",index)

        
        

0 09:00:00
1 13:30:00
2 14:00:00
3 10:00:00
4 14:00:00
5  
6 09:30:00
7 10:00:00
8 10:00:00
9 13:30:00
10 09:00:00
11 14:00:00
12 08:00:00
13 10:00:00
14 10:00:00
15 10:00:00
16 15:00:00
17 15:00:00
18  


In [314]:
for i in range(7):
    
    hwp.move_to_field(f'time{{{{{i}}}}}')
    hwp.insert_text(f"{i}")


In [336]:
# date 데이터 넣기
for index, row in date_column_df.iterrows():
    hwp.move_to_field(f'date{{{{{index}}}}}')
    hwp.insert_text(row['date'])
    print(row['date'])

4. 8.(월)
4. 9.(화)
4. 10.(수)
4. 11.(목)
4. 12. (금)
4. 13.(토)
4. 14.(일)


In [323]:
#date컬럼

for i in range(7):
    


,date
0,4. 8.(월)
1,4. 9.(화)
2,4. 10.(수)
3,4. 11.(목)
4,4. 12. (금)
5,4. 13.(토)
6,4. 14.(일)


In [316]:
#date 제외 데이터 넣기
df_no_date = df.drop(columns=['date'])
hwp.set_pos(15,0,0)
# 셀 전부 선택하는 코드 
hwp.TableCellBlock()
hwp.TableCellBlockExtend()
hwp.TableColEnd()
hwp.TableColPageDown()
hwp.set_cur_field_name("imsi")
hwp.put_field_text("imsi", df_no_date.values.flatten().tolist())


In [82]:
#hwp.MoveToField("date", True, True, False)
#-> 왜인지 모르겠지만 첫번쨰 date로 가진다 ㅇㅇ
hwp.TableSplitCell()
# 표 나누기

True

,time,eventname,place,personnel,department,note
0,09:00:00,중앙재난안전대책본부 영상회의(의료계 파업),재난안전상황실,4,재난안전과,
1,13:30:00,찾아가는 치매조기검진 및 치매 예방교육,외삼1통경로당,16,예방의약과,
2,14:00:00,2024년 유성 데이터 기반 실증 리빙랩 발대식,대회의실,40,미래전략과,
3,10:00:00,나도갈래 과학소풍,대전정보문화산업진흥원,20,교육과학과,
4,14:00:00,행복한 문화학교「박완서의 작품 세계 엿보기」,유성도서관,40,도서관운영과,
5,,,,,,
6,09:30:00,아가맘 바라온 만남 「동화 태교와 태담 태교」,아가랑도서관,10,도서관운영과,
7,10:00:00,상반기 공감 인문학 「임홍택 작가 초청 강연」,구암도서관,50,도서관운영과,
8,10:00:00,독서동아리 역량강화교육,온라인(ZOOM),15,도서관운영과,
9,13:30:00,제44회 장애인의 날 및 장애인종합복지관 개관 19주년 기념행사,유성구장애인종합복지관,800,사회돌봄과,
